# Problem 1: Raw Data Processing

Objective: Ingest and process raw stock market datasets.

Tasks:


1.Download the ETF and stock datasets from the primary dataset available at https://www.kaggle.com/datasets/jacksoncrow/stock-market-dataset.


2.Setup a data structure to retain all data from ETFs and stocks with the following columns:

Symbol: string

Security Name: string

Date: string (YYYY-MM-DD)

Open: float

High: float

Low: float

Close: float

Adj Close: float

Volume: int


3.Convert the resulting dataset into a structured format (e.g. Parquet).

In [4]:
!pip install kaggle

  Using cached kaggle-1.5.13.tar.gz (63 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for kaggle: filename=kaggle-1.5.13-py3-none-any.whl size=77731 sha256=f90436a34009ae6ec819ec5fd46bc9e185ec1073cdc0c839179da2bfa3f44882
  Stored in directory: c:\users\y46ju\appdata\local\pip\cache\wheels\1b\22\79\e73b3e988388377c1cda3385bfa20c1799be101c2e12bbe3dc
Successfully built kaggle


In [5]:
!pip list

Package                       Version
----------------------------- ---------------
alabaster                     0.7.12
anaconda-client               1.11.1
anaconda-navigator            2.4.0
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
astroid                       2.14.2
astropy                       5.1
asttokens                     2.0.5
atomicwrites                  1.4.0
attrs                         22.1.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.11.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
backports.tempfile            1.0
backports.weakref             1.0.post1
bcrypt                        3.2.0
beautifulsoup4                4.11.1
binaryornot                   0.4.4
black                         22.6.0
bleac

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "yyk722" # username from the json file
os.environ['KAGGLE_KEY'] = "18d96fe82c283d0153d782f86e597ac3" # key from the json file

In [4]:
!kaggle datasets download -d jacksoncrow/stock-market-dataset


  0%|          | 0.00/522M [00:00<?, ?B/s]
  0%|          | 1.00M/522M [00:01<11:33, 788kB/s]
  0%|          | 2.00M/522M [00:01<06:05, 1.49MB/s]
  1%|          | 3.00M/522M [00:01<03:56, 2.31MB/s]
  1%|          | 4.00M/522M [00:01<03:01, 3.00MB/s]
  1%|          | 5.00M/522M [00:02<02:33, 3.54MB/s]
  1%|1         | 6.00M/522M [00:02<02:20, 3.85MB/s]
  1%|1         | 7.00M/522M [00:02<02:33, 3.51MB/s]
  2%|1         | 8.00M/522M [00:03<02:59, 3.00MB/s]
  2%|1         | 9.00M/522M [00:03<02:48, 3.19MB/s]
  2%|1         | 10.0M/522M [00:04<05:20, 1.67MB/s]
  2%|2         | 13.0M/522M [00:05<02:47, 3.19MB/s]
  3%|2         | 14.0M/522M [00:05<02:40, 3.33MB/s]
  3%|2         | 15.0M/522M [00:05<02:30, 3.53MB/s]
  3%|3         | 16.0M/522M [00:05<02:29, 3.54MB/s]
  3%|3         | 17.0M/522M [00:06<02:34, 3.43MB/s]
  3%|3         | 18.0M/522M [00:06<02:31, 3.49MB/s]
  4%|3         | 19.0M/522M [00:06<02:30, 3.51MB/s]
  4%|3         | 20.0M/522M [00:07<02:20, 3.76MB/s]
  4%|4         | 21.0

In [5]:
import zipfile

In [6]:
with zipfile.ZipFile('stock-market-dataset.zip', 'r') as zip_file:
    zip_file.extractall('./')

In [28]:
#!pip install pandas
!pip install pyarrow

     ---------------------------------------- 21.5/21.5 MB 4.2 MB/s eta 0:00:00


In [35]:
import pandas as pd
import os
import concurrent.futures
import time

In [31]:
dtypes = {
    'Symbol': str,
    'Security Name': str,
    'Date': str,
    'Open': float,
    'High': float,
    'Low': float,
    'Close': float,
    'Adj Close': float,
    'Volume': int
    }

In [37]:
symbols_valid_meta = pd.read_csv("symbols_valid_meta.csv")

In [38]:
symbols_valid_meta = symbols_valid_meta[['Symbol', 'Security Name', 'ETF']]

In [39]:
symbols_valid_meta_stocks = symbols_valid_meta[symbols_valid_meta.ETF=='N']
symbols_valid_meta_etfs = symbols_valid_meta[symbols_valid_meta.ETF=='Y']

In [40]:
symbols_valid_meta_stocks_symbol = symbols_valid_meta_stocks.Symbol.to_list()
symbols_valid_meta_stocks_security_name = symbols_valid_meta_stocks['Security Name'].to_list()
symbols_valid_meta_etfs_symbol = symbols_valid_meta_etfs.Symbol.to_list()
symbols_valid_meta_etfs_security_name = symbols_valid_meta_etfs['Security Name'].to_list()

In [25]:
os.mkdir('stocks_result')
os.mkdir('etfs_result')

In [ ]:
for i in range(len(symbols_valid_meta_stocks_symbol)):
    df = pd.read_csv('stocks/'+symbols_valid_meta_stocks_symbol[i]+'.csv')
    df['Symbol'] = symbols_valid_meta_stocks_symbol[i]
    df['Security Name'] = symbols_valid_meta_stocks_security_name[i]
    df = df.astype(dtypes)
    df.to_parquet('stocks_result/'+symbols_valid_meta_stocks_symbol[i]+'.parquet')

In [37]:
def process_stock_file(symbol):
    df = pd.read_csv('stocks/'+symbol+'.csv')
    df['Symbol'] = symbol
    df['Security Name'] = symbols_valid_meta_stocks_security_name[symbols_valid_meta_stocks_symbol.index(symbol)]
    df = df.astype(dtypes)
    df.to_parquet('stocks_result/'+symbol+'.parquet')

# Create a thread pool with a maximum threads
max_threads_list = [1, 4, 16, 32, 64]
for max_threads in max_threads_list:
    start_time = time.time()
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
        # Submit each stock symbol to the thread pool for processing
        futures = [executor.submit(process_stock_file, symbol) for symbol in symbols_valid_meta_stocks_symbol]
        # Wait for all threads to complete
        concurrent.futures.wait(futures)
    end_time = time.time()
    execution_time = end_time - start_time
    print("Execution time:", execution_time)

Execution time: 95.2776210308075
Execution time: 47.03400111198425
Execution time: 44.99587678909302
Execution time: 45.331650257110596
Execution time: 47.69900417327881


In [38]:
def process_etf_file(symbol):
    df = pd.read_csv('etfs/'+symbol+'.csv')
    df['Symbol'] = symbol
    df['Security Name'] = symbols_valid_meta_etfs_security_name[symbols_valid_meta_etfs_symbol.index(symbol)]
    df = df.astype(dtypes)
    df.to_parquet('etfs_result/'+symbol+'.parquet')

# Create a thread pool with a maximum threads
max_threads_list = [1, 4, 16, 32, 64]
for max_threads in max_threads_list:
    start_time = time.time()
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
        # Submit each stock symbol to the thread pool for processing
        futures = [executor.submit(process_etf_file, symbol) for symbol in symbols_valid_meta_etfs_symbol]
        # Wait for all threads to complete
        concurrent.futures.wait(futures)
    end_time = time.time()
    execution_time = end_time - start_time
    print("Execution time:", execution_time)

Execution time: 28.3250253200531
Execution time: 16.208886861801147
Execution time: 15.483115673065186
Execution time: 15.678836822509766
Execution time: 15.579127311706543


max threads = 16

# Problem 2: Feature Engineering

Objective: Build some feature engineering on top of the dataset from Problem 1.

Tasks:

Calculate the moving average of the trading volume (Volume) of 30 days per each stock and ETF, and retain it in a newly added column vol_moving_avg.

Similarly, calculate the rolling median and retain it in a newly added column adj_close_rolling_med.

Retain the resulting dataset into the same format as Problem 1, but in its own stage/directory distinct from the first.

(Bonus) Write unit tests for any relevant logic.

In [32]:
!pip install TA-Lib

In [26]:
import unittest
import talib.abstract as ta

In [39]:
os.mkdir('stage')
os.mkdir('stage/stocks_result')
os.mkdir('stage/etfs_result')

In [33]:
# Read in the data
df = pd.read_parquet("stocks_result/A.parquet")
df['close'] = df['Close']
# Convert the Date column to a datetime data type
df["Date"] = pd.to_datetime(df["Date"])

# Set the Date column as the DataFrame's index
df.set_index("Date", inplace=True)

# Calculate the rolling 30-day average of the Volume column
df["vol_moving_avg"] = df["Volume"].rolling(window=30).mean()
df["adj_close_rolling_med"] = df["Adj Close"].rolling(window=30).median()

# RSI
df['rsi'] = ta.RSI(df)
# TEMA - Triple Exponential Moving Average
df["tema"] = ta.TEMA(df, timeperiod=30)

# Drop any rows with missing values
df.dropna(inplace=True)
df.reset_index(inplace=True)
df = df.astype(dtypes)
print(df)

            Date       Open       High        Low      Close  Adj Close  \
0     2000-03-24  84.942772  86.552216  83.154503  85.836906  73.823631   
1     2000-03-27  84.540413  85.479256  80.963875  81.723892  70.286240   
2     2000-03-28  81.545067  87.222816  81.187408  85.211014  73.285332   
3     2000-03-29  85.121605  86.552216  78.683830  80.114449  68.902054   
4     2000-03-30  77.387337  80.472099  70.904861  75.107300  64.595680   
...          ...        ...        ...        ...        ...        ...   
5032  2020-03-26  70.000000  74.449997  69.650002  73.720001  73.532867   
5033  2020-03-27  71.550003  73.209999  70.279999  70.910004  70.730003   
5034  2020-03-30  71.059998  73.180000  71.059998  72.669998  72.669998   
5035  2020-03-31  72.339996  72.800003  70.500000  71.620003  71.620003   
5036  2020-04-01  69.470001  70.230003  68.150002  68.919998  68.919998   

       Volume Symbol                            Security Name      close  \
0     1230500      A  A

In [54]:
def fe_stock_file(symbol):
    df = pd.read_parquet('stocks_result/'+symbol+'.parquet')
    df['close'] = df['Close']
    # Convert the Date column to a datetime data type
    df["Date"] = pd.to_datetime(df["Date"])

    # Set the Date column as the DataFrame's index
    df.set_index("Date", inplace=True)

    # Calculate the rolling 30-day average of the Volume column
    df["vol_moving_avg"] = df["Volume"].rolling(window=30).mean()
    df["adj_close_rolling_med"] = df["Adj Close"].rolling(window=30).median()
    
    # RSI
    df['rsi'] = ta.RSI(df)
    # TEMA - Triple Exponential Moving Average
    df["tema"] = ta.TEMA(df, timeperiod=30)

    # Drop any rows with missing values
    df.dropna(inplace=True)
    df.reset_index(inplace=True)
    df = df.astype(dtypes)
    df.to_parquet('stage/stocks_result/'+symbol+'.parquet')
    
# Create a thread pool with a maximum threads
max_threads_list = [16]
for max_threads in max_threads_list:
    start_time = time.time()
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
        # Submit each stock symbol to the thread pool for processing
        futures = [executor.submit(fe_stock_file, symbol) for symbol in symbols_valid_meta_stocks_symbol]
        # Wait for all threads to complete
        concurrent.futures.wait(futures)
    end_time = time.time()
    execution_time = end_time - start_time
    print("Execution time:", execution_time)

Execution time: 101.15953946113586


In [55]:
def fe_etf_file(symbol):
    df = pd.read_parquet('etfs_result/'+symbol+'.parquet')
    df['close'] = df['Close']
    # Convert the Date column to a datetime data type
    df["Date"] = pd.to_datetime(df["Date"])

    # Set the Date column as the DataFrame's index
    df.set_index("Date", inplace=True)

    # Calculate the rolling 30-day average of the Volume column
    df["vol_moving_avg"] = df["Volume"].rolling(window=30).mean()
    df["adj_close_rolling_med"] = df["Adj Close"].rolling(window=30).median()

    # RSI
    df['rsi'] = ta.RSI(df)
    # TEMA - Triple Exponential Moving Average
    df["tema"] = ta.TEMA(df, timeperiod=30)
    
    # Drop any rows with missing values
    df.dropna(inplace=True)
    df.reset_index(inplace=True)
    df = df.astype(dtypes)
    df.to_parquet('stage/etfs_result/'+symbol+'.parquet')
    
# Create a thread pool with a maximum threads
max_threads_list = [16]
for max_threads in max_threads_list:
    start_time = time.time()
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
        # Submit each stock symbol to the thread pool for processing
        futures = [executor.submit(fe_etf_file, symbol) for symbol in symbols_valid_meta_etfs_symbol]
        # Wait for all threads to complete
        concurrent.futures.wait(futures)
    end_time = time.time()
    execution_time = end_time - start_time
    print("Execution time:", execution_time)

Execution time: 27.9878990650177


# Problem 3: Integrate ML Training

Objective: Integrate an ML predictive model training step into the data pipeline.

You can use the following simple Random Forest model as a reference:

In [62]:
!pip install scikit-learn

     ---------------------------------------- 0.0/8.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/8.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/8.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/8.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/8.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/8.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/8.3 MB 81.9 kB/s eta 0:01:41
     ---------------------------------------- 0.0/8.3 MB 81.9 kB/s eta 0:01:41
     ---------------------------------------- 0.0/8.3 MB 98.5 kB/s eta 0:01:24
     ---------------------------------------- 0.0/8.3 MB 98.5 kB/s eta 0:01:24
     ---------------------------------------- 0.1/8.3 MB 126.1 kB/s eta 0:01:06
     ---------------------------------------- 0.1/8.3 MB 126.9 kB/s eta 0:01:05
     ---------------------------------------- 0.1/8.3 MB 154.3 kB/s eta 0:00:53
      ------------

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [44]:
from pathlib import Path
import pandas as pd

In [56]:
df_list = []
pathlist = Path('./stage/stocks_result').rglob('*.parquet')
for path in pathlist:
    path_in_str = str(path)
    df = pd.read_parquet(path_in_str)
    if 'etfs' in path_in_str:
        df['etf'] = True
    else:
        df['etf'] = False
    df_list.append(df)
    if len(df_list)>10:
        break

pathlist = Path('./stage/etfs_result').rglob('*.parquet')
for path in pathlist:
    path_in_str = str(path)
    df = pd.read_parquet(path_in_str)
    if 'etfs' in path_in_str:
        df['etf'] = True
    else:
        df['etf'] = False
    df_list.append(df)
    if len(df_list)>20:
        break

In [57]:
data = pd.concat(df_list)

In [58]:
data.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Symbol',
       'Security Name', 'close', 'vol_moving_avg', 'adj_close_rolling_med',
       'rsi', 'tema', 'etf'],
      dtype='object')

In [59]:
# Assume `data` is loaded as a Pandas DataFrame
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
# Remove rows with NaN values
data.dropna(inplace=True)

In [60]:
# Select features and target
features = ['vol_moving_avg', 'adj_close_rolling_med', 'etf', 'Symbol', 'rsi', 'tema']
target = ['Volume']

In [9]:
X = data[features]
y = data[target]

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on test data
y_pred = model.predict(X_test)

# Calculate the Mean Absolute Error and Mean Squared Error
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

ValueError: could not convert string to float: 'CW'

In [35]:
mae, mse, rmse

(1186762.5740922473, 4129717606796.904, 2032170.663796942)

In [6]:
!pip install h2o==3.40.0.4

  Using cached h2o-3.40.0.4.tar.gz (177.6 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for h2o: filename=h2o-3.40.0.4-py2.py3-none-any.whl size=177697897 sha256=dd67683c7c6b93afa57ea2e5e066470b1610571dea7c079653c2b1151b7237a8
  Stored in directory: c:\users\y46ju\appdata\local\pip\cache\wheels\5d\cb\0e\dd8dc3921ae87b3657d7f1b83607100fd896107eef31d404c6
Successfully built h2o


In [78]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
     ---------------------------------------- 0.0/177.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/177.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/177.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/177.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/177.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/177.6 MB ? eta -:--:--
     -------------------------------------- 0.0/177.6 MB 100.9 kB/s eta 0:29:21
     -------------------------------------- 0.0/177.6 MB 109.3 kB/s eta 0:27:05
     -------------------------------------- 0.0/177.6 MB 109.3 kB/s eta 0:27:05
     -------------------------------------- 0.1/177.6 MB 142.6 kB/s eta 0:20:45
     -------------------------------------- 0.1/177.6 MB 145.6 kB/s eta 0:20:20
     -------------------------------------- 0.1/177.6 MB 169.3 kB/s eta 0:17:29
  

In [7]:
import h2o
from h2o.automl import H2OAutoML

In [23]:
import pandas as pd

In [8]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 19.0.2+7-44, mixed mode, sharing)
  Starting server from C:\Users\y46ju\anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\y46ju\AppData\Local\Temp\tmpn2cnzh5x
  JVM stdout: C:\Users\y46ju\AppData\Local\Temp\tmpn2cnzh5x\h2o_y46ju_started_from_python.out
  JVM stderr: C:\Users\y46ju\AppData\Local\Temp\tmpn2cnzh5x\h2o_y46ju_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Hong_Kong
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,6 days
H2O_cluster_name:,H2O_from_python_y46ju_ltyt7k
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.910 Gb
H2O_cluster_total_cores:,32
H2O_cluster_allowed_cores:,32
H2O_cluster_status:,"locked, healthy"


In [63]:
h2o_train, h2o_test = train_test_split(data, test_size=0.2, random_state=42)

In [64]:
h2o_train, h2o_test = h2o.H2OFrame(h2o_train), h2o.H2OFrame(h2o_test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [65]:
h2o_train['etf'] = h2o_train['etf'].asfactor()
h2o_train['Symbol'] = h2o_train['Symbol'].asfactor()

h2o_test['etf'] = h2o_test['etf'].asfactor()
h2o_test['Symbol'] = h2o_test['Symbol'].asfactor()

In [66]:
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=features, y=target[0], training_frame=h2o_train)

AutoML progress: |██
11:44:47.691: AutoML: XGBoost is not available; skipping it.

█████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),3/5
# GBM base models (used / total),1/1
# DRF base models (used / total),2/2
# DeepLearning base models (used / total),0/1
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [70]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_BestOfFamily_1_AutoML_2_20230504_114447,1.66536e+06,2.77341e+12,535346,nan,2.77341e+12
StackedEnsemble_AllModels_1_AutoML_2_20230504_114447,1.66676e+06,2.77809e+12,535719,nan,2.77809e+12
DRF_1_AutoML_2_20230504_114447,1.67952e+06,2.82079e+12,520138,2.45145,2.82079e+12
XRT_1_AutoML_2_20230504_114447,1.69541e+06,2.87442e+12,567036,3.46156,2.87442e+12
GBM_grid_1_AutoML_2_20230504_114447_model_5,1.72075e+06,2.96099e+12,539287,nan,2.96099e+12
GBM_4_AutoML_2_20230504_114447,1.72654e+06,2.98094e+12,550673,nan,2.98094e+12
GBM_5_AutoML_2_20230504_114447,1.73915e+06,3.02465e+12,555168,nan,3.02465e+12
GBM_3_AutoML_2_20230504_114447,1.74262e+06,3.03672e+12,549978,nan,3.03672e+12
GBM_2_AutoML_2_20230504_114447,1.74592e+06,3.04825e+12,557838,nan,3.04825e+12
GBM_grid_1_AutoML_2_20230504_114447_model_2,1.76657e+06,3.12078e+12,562678,nan,3.12078e+12


In [71]:
perf = aml.leader.model_performance(h2o_test)
perf

ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 1997886877127.4148
RMSE: 1413466.2631727066
MAE: 508563.8621994182
RMSLE: NaN
Mean Residual Deviance: 1997886877127.4148
R^2: 0.8261375923205657
Null degrees of freedom: 11833
Residual degrees of freedom: 11830
Null deviance: 1.3599393210855389e+17
Residual deviance: 2.3642993303925828e+16
AIC: 368769.134866562

In [72]:
model_path = h2o.save_model(model=aml.get_best_model(), path="./", force=True)
print(model_path)

C:\Users\y46ju\OneDrive\Documents\Python Scripts\riskThinkingWorkSample\experiment\StackedEnsemble_BestOfFamily_1_AutoML_2_20230504_114447


In [9]:
saved_model = h2o.load_model("StackedEnsemble_AllModels_1_AutoML_1_20230504_100516")

In [74]:
h2o_test

Open,High,Low,Close,Adj Close,Volume,Symbol,Security Name,close,vol_moving_avg,adj_close_rolling_med,rsi,tema,etf
34.25,34.353,34.16,34.353,33.7981,1000,ACSI,American Customer Satisfaction ETF,34.353,3136.67,33.2294,61.9764,34.5505,True
65,66.93,64.92,66.46,64.9043,684200,AAP,Advance Auto Parts Inc Advance Auto Parts Inc W/I,66.46,914703,63.1513,66.6579,66.6596,False
4.14,4.24,4.06,4.19,4.05227,17400,AAME,Atlantic American Corporation - Common Stock,4.19,13640,3.8395,58.5827,4.19033,False
0,5.15,5.05,5.05,4.21467,9000,AAME,Atlantic American Corporation - Common Stock,5.05,10490,4.17294,52.1981,5.05463,False
2.48,2.63,2.45,2.63,2.43041,3700,AAME,Atlantic American Corporation - Common Stock,2.63,7040,2.37035,55.4279,2.63888,False
78.9,79.12,77.36,77.67,76.4561,763300,AAP,Advance Auto Parts Inc Advance Auto Parts Inc W/I,77.67,1.08921e+06,76.1214,53.8358,77.9966,False
19.0918,19.224,18.6713,19.0918,11.6814,2.9325e+06,AA,Alcoa Corporation Common Stock,19.0918,1.45115e+06,12.1299,35.5301,19.6343,False
51.42,51.58,51.13,51.43,49.9479,1.2594e+06,A,"Agilent Technologies, Inc. Common Stock",51.43,1.99501e+06,49.5546,57.6547,52.1132,False
54.5241,54.6683,53.2745,54.2838,42.2394,1.7064e+06,AA,Alcoa Corporation Common Stock,54.2838,1.68292e+06,47.6367,35.0913,57.0213,False
40.07,40.14,39.64,39.65,38.0902,2.1726e+06,A,"Agilent Technologies, Inc. Common Stock",39.65,2.21246e+06,36.5099,58.1535,39.4062,False


In [10]:
frame = h2o.H2OFrame({"Symbol":"A", "etf":False, "rsi":58.1535, "tema":39.4062, "vol_moving_avg":2212460, "adj_close_rolling_med":36.5099})

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [11]:
frame

Symbol,etf,rsi,tema,vol_moving_avg,adj_close_rolling_med
A,False,58.1535,39.4062,2.21246e+06,36.5099


In [14]:
result = saved_model.predict(frame)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [28]:
result.as_data_frame()['predict'][0]

2880320.593882378